# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait


# Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')


In [3]:
driver.find_element_by_id('mcrbutton').click() 
driver.find_element_by_id('mcrdata').send_keys('006179570C') 
button = driver.find_element_by_id('submit3') 
driver.execute_script("arguments[0].scrollIntoView(true)", button) 
button.click()

In [8]:
time.sleep(1)
company_info = re.split(":   ",driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody').text)
company_info[1]
re.split(" /", company_info[3])[0]
company_info[4]

company_details = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]').text
re.findall("\((.*)\)", company_details)[0]


address = re.split(":\n", company_details)[-1]
str.replace(address, "\n", " ")

'13619 BRETT JACKSON RD. FORT WORTH, TX. 76179'

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [9]:

driver.get('https://www.tdlr.texas.gov/tools_search/')

driver.find_element_by_id('mcrbutton').click() 
driver.find_element_by_id('mcrdata').send_keys('006179570C') 
button = driver.find_element_by_id('submit3') 
driver.execute_script("arguments[0].scrollIntoView(true)", button) 
button.click()


time.sleep(1)
company_info = re.split(":   ",driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody').text)
print(company_info[1]) 
business = company_info[1]
print(re.split(" /", company_info[3])[0]) 
owner = re.split(" /", company_info[3])[0]
print(company_info[4]) 
phone = company_info[4]

company_details = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]').text
print(re.findall("\((.*)\)", company_details)[0]) 
status = re.findall("\((.*)\)", company_details)[0]

address = re.split(":\n", company_details)[-1]
print(str.replace(address, "\n", " "))
address = str.replace(address, "\n", " ")

B.D. SMITH TOWING DBA
BRANDT SMITH
8173330706
Expired
13619 BRETT JACKSON RD. FORT WORTH, TX. 76179


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.
### Open up `trucks-subset.csv` and save it into a dataframe

In [11]:
df  =  pd.read_csv('trucks-subset.csv')
df.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


(3, 1)

## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [12]:

def  get_towtruck(row):

    
    url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='+row['TDLR Number']
    return url

In [13]:
df.apply(get_towtruck, axis=1)

0    https://www.tdlr.texas.gov/tools_search/mccs_d...
1    https://www.tdlr.texas.gov/tools_search/mccs_d...
2    https://www.tdlr.texas.gov/tools_search/mccs_d...
dtype: object

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_d...
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_d...
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_d...


### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [16]:
def get_towtruck(row):
    driver.get('https://www.tdlr.texas.gov/tools_search/')
    driver.find_element_by_id('mcrbutton').click() 
    driver.find_element_by_id('mcrdata').send_keys(row['TDLR Number']) 
    button = driver.find_element_by_id('submit3') 
    driver.execute_script("arguments[0].scrollIntoView(true)", button)
    button.click()
    
    url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='+row['TDLR Number']
    
    return pd.Series({
            'url': url
            
        })

In [18]:
pd.set_option('display.max_colwidth', -1)
url_df = df.apply(get_towtruck, axis=1).join(df)
url_df

,url,TDLR Number
0,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C,006507931C
1,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C,006179570C
2,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C,006502097C


In [21]:



def  get_towtruckget(row):
        
    driver.get('https://www.tdlr.texas.gov/tools_search/')
    
    
    driver.find_element_by_id('mcrbutton').click() 
    driver.find_element_by_id('mcrdata').send_keys(row['TDLR Number'])
    print(row['TDLR Number'])
    button = driver.find_element_by_id('submit3') 
    driver.execute_script("arguments[0].scrollIntoView(true)", button) 
    button.click()
    url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='+row['TDLR Number']

    try:
        
        time.sleep(1)
        company_info = re.split(":   ",driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody').text)
        print(company_info[1]) 
        business = company_info[1]
        print(re.split(" /", company_info[3])[0]) 
        owner = re.split(" /", company_info[3])[0]
        print(company_info[4]) 
        phone = company_info[4]
        
        company_details = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]').text
        print(re.findall("\((.*)\)", company_details)[0]) #status
        status = re.findall("\((.*)\)", company_details)[0]
        
        address = re.split(":\n", company_details)[-1]
        print(str.replace(address, "\n", " "))
        address = str.replace(address, "\n", " ")
        
        return pd.Series({
                'business': business,
                'owner': owner,
                'phone': phone,
                'status': status,
                'address': address,
                'url': url
                
            })
    except:
        print("no record")
        return pd.Series({
                'business': 'no record',
                'owner': 'no record',
                'phone': 'no record',
                'status': 'no record',
                'address': 'no record',
                'url': url
                
            })

In [22]:
all_df = df.apply(get_towtruck, axis=1).join(df)

## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [23]:
all_df.to_csv("tow-trucks-extended.csv", index=False)

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

### Save your dataframe as a CSV named `tow-trucks-extended.csv`

### Re-open your dataframe to confirm you didn't save any extra weird columns

## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**